In [1]:
import pandas as pd
import numpy as np

In [10]:
%%time
train_df = pd.read_csv('tap_fun_train.csv')
test_df = pd.read_csv('tap_fun_test.csv')

Wall time: 20.8 s


In [13]:
def cleaning(data):
    data['register_time'] = data.register_time.index
    return data

In [14]:
y = train_df.prediction_pay_price
train_df.drop(['prediction_pay_price'],axis=1,inplace = True)
train_df = cleaning(train_df)
test_df = cleaning(test_df)

In [15]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
x_train = scaler.fit_transform(train_df)
x_test = scaler.transform(test_df)

In [5]:
# from sklearn.model_selection import train_test_split
# x_train, x_test, y_train, y_test = train_test_split(train_df, y, test_size = 0.2)

### Linear Regression

In [8]:
%%time
from sklearn.linear_model import LinearRegression
reg = LinearRegression()
reg.fit(x_train, y)
preds = reg.predict(test_df)

Wall time: 18.7 s


### ElasticNet

In [16]:
%%time
from sklearn.linear_model import ElasticNet
reg = ElasticNet()
reg.fit(x_train, y)
preds = reg.predict(x_test)

Wall time: 1min 1s


### Ridge Regression

In [20]:
%%time
from sklearn.linear_model import Ridge
reg2 = Ridge(alpha=1.0)
reg2.fit(x_train, y)
preds = reg2.predict(x_test)

Wall time: 3.48 s


### Output Processing

In [21]:
## output processing
preds[preds<0.5] = 0
abb = pd.DataFrame(preds)
abb[(abb[0]>0) & (abb[0]<1)] = 0.99
sub = pd.DataFrame({'user_id':test_df.user_id,'prediction_pay_price': abb[0]},columns=['user_id','prediction_pay_price'])
sub.to_csv('output_processed_Ridge.csv',index=None)

In [23]:
sub.describe()

,user_id,prediction_pay_price
count,8.289340e+05,828934.000000
mean,1.811513e+06,3.239284
std,8.229500e+05,85.946625
min,1.493300e+04,0.000000
25%,1.286373e+06,0.000000
50%,2.012872e+06,0.000000
75%,2.481669e+06,0.990000
max,3.176543e+06,45919.468251
